# R OBIS

This notebook was adapted from [a manual on robis](https://obis.org/manual/accessr/) on the [obis](obis.org) site. Some of the mapping functions weren't working properly, so we've put in some simple ggplot mapping and plotting examples.

## Loading robis

Installing the robis packages requires devtools. Use the following (this code won't run within the notebook.)

In [1]:
installed <- rownames(installed.packages())
if ( !("robis" %in% installed) ){
   if ( !("remotes" %in% installed) )install.packages("remotes")
   remotes::install_github("iobis/robis")
}

Using github PAT from envvar GITHUB_PAT




KernSmooth   (NA -> 2.23-17 ) [CRAN]
class        (NA -> 7.3-17  ) [CRAN]
e1071        (NA -> 1.7-3   ) [CRAN]
BH           (NA -> 1.72.0-3) [CRAN]
commonmark   (NA -> 1.7     ) [CRAN]
fastmap      (NA -> 1.0.1   ) [CRAN]
sourcetools  (NA -> 0.1.7   ) [CRAN]
xtable       (NA -> 1.8-4   ) [CRAN]
httpuv       (NA -> 1.5.4   ) [CRAN]
units        (NA -> 0.6-7   ) [CRAN]
classInt     (NA -> 0.4-3   ) [CRAN]
shiny        (NA -> 1.5.0   ) [CRAN]
systemfonts  (NA -> 0.2.3   ) [CRAN]
gdtools      (NA -> 0.2.2   ) [CRAN]
svglite      (NA -> 1.2.3.2 ) [CRAN]
brew         (NA -> 1.0-6   ) [CRAN]
webshot      (NA -> 0.5.2   ) [CRAN]
sf           (NA -> 0.9-5   ) [CRAN]
satellite    (NA -> 1.0.2   ) [CRAN]
leafpop      (NA -> 0.0.5   ) [CRAN]
leafem       (NA -> 0.1.3   ) [CRAN]
miniUI       (NA -> 0.1.1.1 ) [CRAN]
mapview      (NA -> 2.7.8   ) [CRAN]
leafpm       (NA -> 0.1.0   ) [CRAN]
leaflet.e... (NA -> 1.0.0   ) [CRAN]
mapedit      (NA -> 0.6.0   ) [CRAN]
httpcache    (NA -> 1.1.0   ) [CRAN]


Installing 27 packages: KernSmooth, class, e1071, BH, commonmark, fastmap, sourcetools, xtable, httpuv, units, classInt, shiny, systemfonts, gdtools, svglite, brew, webshot, sf, satellite, leafpop, leafem, miniUI, mapview, leafpm, leaflet.extras, mapedit, httpcache



ERROR: Error: Failed to install 'robis' from GitHub:
  (converted from warning) installation of package ‘systemfonts’ had non-zero exit status


Once installed, load the robis package.

In [ ]:
library(robis)

## Fetching occurences

OBIS contains global location data for marine taxa. 

To fetch all occurrences for a taxon, pass a taxon name or vector of taxon names to the occurrence() function. The names() function will give a list of the column names, so you know what data is included. You can see that there is quite a lot of information for each sample.

In [ ]:
E.glacialis <- occurrence("Eubalaena glacialis")
names(E.glacialis)